In [ ]:
# Notebook: 911 Demo - Data Preprocessing and Cleaning
# Author: Thomas Purk
# Date: 2025-03-14
# Reference: https://www.kaggle.com/datasets/mchirico/montcoalert

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 3. Data Preprocessing & Cleaning
- Handle **missing data**, **inconsistent formats**, and **outliers**.
- **Feature engineering**: Create new features, encode categorical variables, normalize/scale features.
- Split data into **training, validation, and test sets**.

---

In [ ]:
# Load the data and display a list of columns
df_911 = pd.read_csv('/kaggle/input/montcoalert/911.csv')
df_911.info()

In [ ]:
df_911.head()

### 3.1 Inspect Each Feature

In [ ]:
# Inspect the lat feature
print("### lat ###")
df_911['lat'].describe()

**Notes on lat**
- WGS84 Lat ranges is about 39.92845686753457,  40.49685656154363, 
- Zero latitude is the equator. These are not valid values for locations in PA
- Postive 51 degrees latitude is too far north, in Quebec Canada
- The mean around 40 degrees is about right for Montgomery County PA
- So there are some incorrect values in this data

In [ ]:
# Inspect the lat feature
print("### lng ###")
df_911['lng'].describe()

**Notes on lng**
- WGS84 lng ranges is about -75.72962906989237, -74.99629164995669
- Eastern Hemisphere always has negative longitudes
- So there are some incorrect values in this data

In [ ]:
# Inspect the desc feature
print("### desc ###")
df_911['desc'].describe()

In [ ]:
# Inspect split of desc feature
desc_split = df_911['desc'].str.split(';', expand=True)
display(desc_split.info())
display(desc_split.head())

**Notes on desc**
- string feature
- appears to contain several values
- Values may be repeated in other features
- But Station name may be unique

In [ ]:
# CLEAN UP "desc" - Remove substrings from desc that are repeated in other fields

# The goal is to find valuable data in the desc field if any

# Copy the data
df_911_v1 = df_911.copy()

# Loop through rows using iterrows()
for index, row in df_911_v1.iterrows():
    # Remove addr string
    # Have some bad addr value that are just integers
    if(not row['addr'].isdigit()):
        new_value = row['desc'].replace(row['addr'], '')

    
    # Remove twp string
    new_value = new_value.replace(str(row['twp']), '')
    # The Desc feature contains timestamps Example: "2015-12-10 @ 17:10:52"
    # But the timeStamp feature is formated as  "2015-12-10 17:10:52"
    # So split the timeStamp by ' ' and remove each part seperately
    ts_parts = row['timeStamp'].split(' ')
    new_value = new_value.replace(ts_parts[0], '')
    new_value = new_value.replace(ts_parts[1], '')
    # clean up delimiters
    new_value = new_value.replace(';','')
    new_value = new_value.replace('@','')
    new_value = new_value.replace('-','')
    new_value = new_value.replace(':','')
    # "Station" and "STA" are not needed
    new_value = new_value.replace('Station','')
    new_value = new_value.replace('STA','')
    new_value = new_value.strip()
    # Write the remaining string as a new feature
    df_911_v1.at[index, 'station'] = new_value

In [ ]:
print(f'desc Null count: {df_911_v1.desc.isnull().sum()}')
print(f'desc "" count: {(df_911_v1.desc == "").sum()}')

In [ ]:
s = df_911_v1[df_911_v1['desc'] != ''][['desc','twp']].value_counts()
s

In [ ]:
for value, count in df_911_v1[['station', 'addr']].value_counts().items():
    print(f"{value}: {count}")

In [ ]:
df_911[df_911_v1['desc'] == '1:05:44']

### 4. **Exploratory Data Analysis (EDA)**
- Analyze data distributions, correlations, and patterns.
- Visualize data (e.g., histograms, scatter plots, heatmaps).
- Understand relationships and potential biases in the dataset.

---

In [ ]:
df['twp'].value_counts()

In [ ]:
keys = df['twp'].value_counts().keys()
keys = keys.sort_values()
for k in keys:
    print(k)

In [ ]:
len(df)